https://fivethirtyeight.com/features/can-you-solve-the-vexing-vexillology/

**Riddler Classic**
The New York Times recently launched some new word puzzles, one of which is Spelling Bee. In this game, seven letters are arranged in a honeycomb lattice, with one letter in the center. Here’s the lattice from December 24, 2019:

_Spelling Bee screenshot, with the required letter G, and the additional letters L, A, P, X, M and E._

The goal is to identify as many words that meet the following criteria:

- The word must be at least four letters long.
- The word must include the central letter.
- The word cannot include any letter beyond the seven given letters.
Note that letters can be repeated. For example, the words GAME and AMALGAM are both acceptable words. Four-letter words are worth 1 point each, while five-letter words are worth 5 points, six-letter words are worth 6 points, seven-letter words are worth 7 points, etc. Words that use all of the seven letters in the honeycomb are known as “pangrams” and earn 7 bonus points (in addition to the points for the length of the word). So in the above example, MEGAPLEX is worth 15 points.

Which seven-letter honeycomb results in the highest possible game score? To be a valid choice of seven letters, no letter can be repeated, it must not contain the letter S (that would be too easy) and there must be at least one pangram.

For consistency, please use this word list to check your game score.
https://norvig.com/ngrams/enable1.txt **(NB: saved locally)**
This is different from The New York Times word list, but please stick to this list for the purposes of the puzzle.

In [1]:
# First step: import the word list

# Note: need to include na_filter=False because the word list includes
# "nan" and "null" which will otherwise be imported by pandas as floats
# https://stackoverflow.com/questions/33952142/prevent-pandas-from-interpreting-na-as-nan-in-a-string
import pandas as pd
words = pd.read_csv('wordlist.txt', header=None, names=['word'], dtype={'word': str}, na_filter=False)
words.shape  # 172820 raw

(172820, 1)

In [2]:
# Confirm everything read ok - should be an empty frame
words[words.word.apply(type) != str]

,word


In [3]:
# Break every word into a (frozen)set of its unique letters and store that in a separate col
words['ltrset'] = words.word.apply(frozenset)

In [4]:
# Function to score words, per above:
# Four-letter words are worth 1 point each, while five-letter words are worth 5 points,
# six-letter words are worth 6 points, seven-letter words are worth 7 points, etc.
# Words that use all of the seven letters in the honeycomb are known as “pangrams” and
# earn 7 bonus points (in addition to the points for the length of the word)
# (Also: no 'S' allowed per the puzzle parameters)
def sb_score(word, ltrset):
    wl = len(word)
    sl = len(ltrset)
    
    if wl < 4 or sl > 7 or 's' in ltrset:
        return 0
    elif wl == 4:
        return 1
    elif sl < 7:
        return wl
    else:
        return wl + 7

In [5]:
# Score every word in the list and add to the df
words['sbscore'] = words.apply(lambda x: sb_score(x.word, x.ltrset), axis=1)

In [6]:
# Words with score 0 can be removed from the word list so there's less to search
words.drop(words[words.sbscore == 0].index, inplace=True)  #128,235 rows

In [7]:
# Find all valid pangrams -- i.e., 7-letter sets
pangrams = words[words.ltrset.apply(len) == 7]  # 14,741 rows
len(pangrams)

14741

In [29]:
# The unique sets of letters (there are some duplicates) formed by all pangrams are our candidates for the answer
candidates = pd.DataFrame(index=pangrams.ltrset.unique())

# Add columns to hold the max score for each set and the "required" letter used to produce that score
candidates['top_score'] = 0
candidates['best_letter'] = None
candidates.head()
len(candidates)

7986

In [9]:
# For each candidate set, find all possible words from the word list, then test each letter in the set
# to see which letter yields the highest score when required (i.e., in the middle of the hexagon) 
for ltrset in candidates.index:
    # For each candidate set, find all matching words from the list
    set_matches = words[words.ltrset.apply(lambda s: s.issubset(ltrset))]

    # Calculate score with each letter required to see which letter produces the highest score for this set
    top_score = 0
    best_letter = None

    for ltr in ltrset:
        score = set_matches[set_matches.ltrset.apply(lambda s: ltr in s)].sbscore.sum()

        if score > top_score:
            top_score = score
            best_letter = ltr

    # Store the letter that produces the max score for each set along with the score
    candidates.at[ltrset, 'best_letter'] = best_letter
    candidates.at[ltrset, 'top_score'] = top_score

In [14]:
# Finally, preview the results. Ordering by the largest score should produce the solution
candidates.nlargest(10, 'top_score')

,top_score,best_letter
"(a, e, i, g, r, t, n)",3898,r
"(a, e, i, d, r, t, n)",3672,e
"(a, e, i, g, d, r, n)",3271,e
"(a, e, i, d, r, t, l)",3194,e
"(a, e, d, r, t, n, p)",3060,e
"(c, a, e, i, d, r, t)",3023,e
"(a, e, l, d, r, t, p)",3005,e
"(a, e, g, d, r, t, n)",3001,e
"(a, e, i, r, t, n, l)",2983,e
"(c, a, e, i, r, t, n)",2972,t


In [15]:
# Cross check the score by checking possible words
top_row = candidates.nlargest(1, 'top_score')
top_set = top_row.index[0]
top_letter = top_row.best_letter[0]
top_score = top_row.top_score[0]

test_matches = words[words.ltrset.apply(lambda s: s.issubset(top_set)) & words.ltrset.apply(lambda s: top_letter in s)]
test_score = test_matches.sbscore.sum()

assert(test_score == top_score)
print(test_score, "==", top_score)

assert(test_score == candidates.top_score.max())

3898 == 3898


In [28]:
# Print the answer from above
print("Top set is", str.upper(', '.join(top_set)), "with", str.upper(top_letter), "in the middle.")


Top set is A, E, I, G, R, T, N with R in the middle.


In [18]:
# Just for kicks, here's a function to produce all answers for any set of letters
def solve_bee(full_set, reqd_ltr):
    return words[words.ltrset.apply(lambda s: reqd_ltr in s and s.issubset(full_set))]

In [19]:
solve_bee({'f', 'w', 'o', 'd', 'l', 'a', 't'}, 't')

,word,ltrset,sbscore
2524,afloat,"(a, o, f, t, l)",6
2526,afoot,"(f, a, t, o)",5
3843,allot,"(l, a, t, o)",5
3978,aloft,"(a, o, f, t, l)",5
4110,alto,"(l, a, t, o)",1
9224,atlatl,"(l, a, t)",6
9238,atoll,"(l, a, t, o)",5
36037,daft,"(d, f, a, t)",1
36469,data,"(d, a, t)",1
36498,dato,"(d, a, t, o)",1
